# Obtaining Your Data - Lab

## Introduction
In this lab you'll practice your munging and transforming skills in order to load in your data to solve a regression problem.

## Objectives
You will be able to:
* Understand the ETL process and the steps it consists of
* Understand the challenges of working with data from multiple sources 

## Task Description

You just got hired by Lego! Your first project is going to be to develop a pricing algorithm to help set a target price for new lego sets that are released to market. To do this, you're first going to need to start mining the company database in order to collect the information you need to develop a model.

Start by investigating the database stored in lego.db and joining the tables into a unified dataset!

> **Hint:** use this sql statement to preview the tables in an unknown database:
```sql
SELECT name FROM sqlite_master
             WHERE type='table'
             ORDER BY name;
```

In [3]:
!ls

CONTRIBUTING.md  economy_data.csv  index.ipynb	LICENSE.md  Walmart.db
data.sqlite	 images		   lego.db	README.md


In [2]:
# Your code here
import sqlite3
import pandas as pd

#Create a connection
con = sqlite3.connect('lego.db')
#Create a cursor
cur = con.cursor()
#Select some data
cur.execute("""SELECT name FROM sqlite_master
             WHERE type='table'
             ORDER BY name;""")
df = pd.DataFrame(cur.fetchall())
df.tail()

,0
0,product_details
1,product_info
2,product_pricing
3,product_reviews


In [3]:
cur.execute("""SELECT * FROM product_details;""")
dfPD = pd.DataFrame(cur.fetchall())
dfPD.tail()

,0,1,2,3
739,853638,blah,blahblah,THE LEGO® BATMAN MOVIE
740,2000409,blah,blahblah,SERIOUS PLAY®
741,2000414,blah,blahblah,SERIOUS PLAY®
742,2000430,blah,blahblah,SERIOUS PLAY®
743,2000431,blah,blahblah,SERIOUS PLAY®


In [4]:
cur.execute("""SELECT * FROM product_info;""")
dfPI = pd.DataFrame(cur.fetchall())
dfPI.tail()

,0,1,2,3
739,853638,7+,61,THE LEGO® BATMAN MOVIE Batman™ Minifigure Coll...
740,2000409,6+,4900,Window Exploration Bag
741,2000414,6+,219,Starter Kit
742,2000430,6+,2631,Identity and Landscape Kit
743,2000431,6+,2455,Connections Kit


In [5]:
cur.execute("""SELECT * FROM product_pricing;""")
dfPP = pd.DataFrame(cur.fetchall())
dfPP.tail()

,0,1,2
10865,70903,PT,42.6878
10866,70910,PT,30.4878
10867,70901,PT,30.4878
10868,70913,PT,18.2878
10869,70900,PT,18.2878


In [6]:
cur.execute("""SELECT * FROM product_reviews;""")
dfPR = pd.DataFrame(cur.fetchall())
dfPR.tail()

,0,1,2,3,4,5
739,853638,19.0,2.1,Very Easy,2.9,2.8
740,2000409,2.0,5.0,Easy,5.0,5.0
741,2000414,10.0,3.9,Easy,3.9,3.3
742,2000430,7.0,4.7,Easy,4.6,3.6
743,2000431,8.0,3.7,Average,3.4,2.9


In [8]:
conn = sqlite3.connect('lego.db')
c = conn.cursor()
response = c.execute("""SELECT name FROM sqlite_master
             WHERE type='table'
             ORDER BY name;
          """).fetchall()
response

[('product_details',),
 ('product_info',),
 ('product_pricing',),
 ('product_reviews',)]

In [9]:
for item in response:
    table = item[0]
    length = c.execute("""SELECT count(*) from {};""".format(table)).fetchall()
    results = c.execute("""SELECT * from {} limit 5;""".format(table)).fetchall()
    df = pd.DataFrame(results)
    df.columns = [x[0] for x in c.description]
    print(table, length, '\n', df, '\n\n')


product_details [(744,)] 
    prod_id prod_desc prod_long_desc theme_name
0      630      blah       blahblah    Classic
1     2304      blah       blahblah     DUPLO®
2     7280      blah       blahblah       City
3     7281      blah       blahblah       City
4     7499      blah       blahblah       City 


product_info [(744,)] 
    prod_id  ages  piece_count                         set_name
0      630    4+            1                  Brick Separator
1     2304  1½-5            1     LEGO® DUPLO® Green Baseplate
2     7280  5-12            2      Straight & Crossroad Plates
3     7281  5-12            2  T-Junction & Curved Road Plates
4     7499  5-12           24     Flexible and Straight Tracks 


product_pricing [(10870,)] 
    prod_id country list_price
0    75823      US      29.99
1    75822      US      19.99
2    75821      US      12.99
3    21030      US      99.99
4    21035      US      79.99 


product_reviews [(744,)] 
    prod_id  num_reviews  play_star_rating re

In [10]:
cmd = """select * from product_info
                  join product_details
                  using(prod_id)
                  join product_pricing
                  using(prod_id)
                  join product_reviews
                  using(prod_id);"""
result = c.execute(cmd).fetchall()
df = pd.DataFrame(result)
df.columns = [x[0] for x in c.description]
print(len(df))
df.head()

10870


,prod_id,ages,piece_count,set_name,prod_desc,prod_long_desc,theme_name,country,list_price,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,75823,6-12,277,Bird Island Egg Heist,blah,blahblah,Angry Birds™,US,29.99,2.0,4.0,Average,4.5,4.0
1,75822,6-12,168,Piggy Plane Attack,blah,blahblah,Angry Birds™,US,19.99,2.0,4.0,Easy,5.0,4.0
2,75821,6-12,74,Piggy Car Escape,blah,blahblah,Angry Birds™,US,12.99,11.0,4.3,Easy,4.3,4.1
3,21030,12+,1032,United States Capitol Building,blah,blahblah,Architecture,US,99.99,23.0,3.6,Average,4.6,4.3
4,21035,12+,744,Solomon R. Guggenheim Museum®,blah,blahblah,Architecture,US,79.99,14.0,3.2,Challenging,4.6,4.1


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10870 entries, 0 to 10869
Data columns (total 14 columns):
prod_id              10870 non-null int64
ages                 10870 non-null object
piece_count          10870 non-null int64
set_name             10870 non-null object
prod_desc            10870 non-null object
prod_long_desc       10870 non-null object
theme_name           10870 non-null object
country              10870 non-null object
list_price           10870 non-null object
num_reviews          9449 non-null float64
play_star_rating     9321 non-null float64
review_difficulty    10870 non-null object
star_rating          9449 non-null float64
val_star_rating      9301 non-null float64
dtypes: float64(4), int64(2), object(8)
memory usage: 1.2+ MB


## Summary
Nice work! You're working more and more independently through the workflow and ensuring data integrity!